In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy import units as u
from astropy.io import fits
#silence DEBUG and WARNINGs
import logging
logging.getLogger('poppy').setLevel(logging.ERROR)
# MagAO-X function definitions
from model_kit import magaoxFunctions as mf
# for writing files
from datetime import date
print('Last date this notebook was ran: {0}'.format(date.today()))

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Last date this notebook was ran: 2020-08-04


In [2]:
# load the information
fr_parm = {'wavelength': 0.9e-6 * u.m,
           'npix': 512, # sample size
           'beam_ratio': 0.25, # oversample
           'surf_off': True} 

mf.build_vapp_mwfs(fr_parm=fr_parm)

In [3]:
%%time
# load the information
fr_parm = {'wavelength': 656.3e-9 * u.m,
           'npix': 512, # sample size
           'beam_ratio': 0.25, # oversample
           'surf_off': True}

mf.build_vapp_mwfs(fr_parm=fr_parm)

CPU times: user 36.2 s, sys: 9.98 s, total: 46.2 s
Wall time: 17.2 s


In [2]:
# reference for old version that used the csv file's vAPP radius value
# this was scrapped in favor of more accurate version calculating pixelscale

def build_vapp_mwfs(fr_parm):
    wavelen = np.round(fr_parm['wavelength'].to(u.nm).value).astype(int)
    br = int(1/fr_parm['beam_ratio'])
    parm_name = '{0:3}_{1:1}x_{2:3}nm'.format(fr_parm['npix'], br, wavelen)

    # load the CSV prescription values
    home_dir = '/home/jhen/XWCL/code/MagAOX/' # change for exao0
    data_dir = home_dir + 'data/'
    rx_loc = 'rxCSV/rx_magaox_NCPDM_sci_{0}.csv'.format(parm_name)
    rx_sys = mf.makeRxCSV(data_dir+rx_loc)

    # acquiring csv numerical values for specifically named optics
    for t_optic, test_opt in enumerate(rx_sys):
        if test_opt['Name'] == 'vAPP-trans':
            vapp_diam = test_opt['Radius_m']*2*u.m

    # vAPP file information
    vAPP_pixelscl = vapp_diam.value/fr_parm['npix'] # direct from csv data file
    vAPP_folder = data_dir+'coronagraph/'
    vAPP_trans_filename = 'vAPP_trans_2PSF_{0}'.format(parm_name)
    vAPP_posOPD_filename = 'vAPP_opd_2PSF_{0}_posPhase'.format(parm_name)
    vAPP_negOPD_filename = 'vAPP_opd_2PSF_{0}_negPhase'.format(parm_name)

    # Call in vAPP data
    vapp_phase_grating_data = fits.open(vAPP_folder+'MagAOX_vAPP_512x512.fits')[0].data
    vapp_aperture_data = fits.open(vAPP_folder+'MagAOX_pupil_512x512.fits')[0].data

    # Calculate the transmissive mask
    mf.writeTRANSfile(vapp_aperture_data, vAPP_pixelscl, vAPP_folder + vAPP_trans_filename + '.fits')

    # Calculate the positive phase OPD and write the file
    vapp_2psf_opd_posPhase = 1*(fr_parm['wavelength'].value/(2*np.pi))*vapp_phase_grating_data*vapp_aperture_data
    mf.writeOPDfile(vapp_2psf_opd_posPhase, vAPP_pixelscl, vAPP_folder + vAPP_posOPD_filename + '.fits')

    # Calculate the negative phase OPD and write the file
    vapp_2psf_opd_negPhase = -1*vapp_2psf_opd_posPhase
    mf.writeOPDfile(vapp_2psf_opd_negPhase, vAPP_pixelscl, vAPP_folder + vAPP_negOPD_filename + '.fits')